In [1]:
import numpy as np
import matplotlib.pyplot as plt


import Code.utils as utils
import Code.environment.settings as settings
from Code.environment.Environment import Environment
from Code.GreedyLearner import GreedyLearner
from Code.environment.Customer import Customer
from Code.UCBLearner import UCBLearner
from Code.TSLearner import TSLearner
from Code.UCBLearner4 import UCBLearner4
from Code.TSLearner4 import TSLearner4
from Code.UCBLearner5 import UCBLearner5
from Code.TSLearner5 import TSLearner5

In [2]:
cust_json = "customer_classes.json"
today_customer = 50
var_cust = 0
p_lambda = 1.0
graph_json = "business_full_graph.json"

arms = settings.prices
environment = Environment(cust_json,today_customer,var_cust,p_lambda, graph_json, arms)
clairvoyant_arms, clairvoyant_reward, exp_c= environment.estimate_clairvoyant()
print("\n", clairvoyant_arms, clairvoyant_reward, exp_c)

loading: [==============================] 99.6%
 [0, 2, 0, 0, 0] 1869.0156000000002 37.380312


In [3]:
n_arms = 4
n_products = 5 

feature1 = 0
feature2 = 0

customer = Customer(feature1, feature2)

TIME_HORIZON = 50
NUM_ITERATION = 1

# TODO: we should put the customer argument as optional,since greedy learner has no need of customer(no simulation)
rewards_iterations = []
for i in range(NUM_ITERATION):
    gr_learner = GreedyLearner(n_arms, n_products, customer, graph_json, arms)

    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = gr_learner.select_superarm()
        report = environment.round(super_arm)
        gr_learner.update(super_arm.copy(), report)
        prices = [arms[p][a] for p, a in enumerate(super_arm)]
    rewards = gr_learner.get_rewards()
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape


loading: [                              ] 0.0%
loading: [======                        ] 20.0%
loading: [============                  ] 40.0%
loading: [==================            ] 60.0%
loading: [========================      ] 80.0%

(1, 50)

In [4]:
#unknown param
c_graph = np.zeros((n_products,n_products))

customer2 = Customer(0,0, environment.get_aggregate_alphas(), environment.get_aggregate_num_prods_distribution(), click_graph=c_graph)

In [5]:
rewards_iterations = []
for i in range(NUM_ITERATION):
    ucb5_learner = UCBLearner5(n_arms, n_products, customer2, graph_json, arms)
    print('Iteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ucb5_learner.select_superarm()
        report = environment.round(super_arm)
        ucb5_learner.update(super_arm.copy(), report)
    rewards = ucb5_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

Iteration # 1
loading: [                              ] 0.0%[[0.  0.  0.  0.  0. ]
 [0.2 0.  1.  0.  0. ]
 [0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.  1. ]
 [1.  0.5 0.  0.  0. ]]
[[0.         0.         0.         0.         0.        ]
 [0.11111111 0.         0.77777778 0.         0.        ]
 [0.         0.6        0.         0.6        0.        ]
 [0.         0.         0.25       0.         1.        ]
 [1.         0.5        0.         0.         0.        ]]
[[0.         1.         0.         0.         1.        ]
 [0.21428571 0.         0.71428571 0.         0.        ]
 [0.         0.75       0.         0.625      0.        ]
 [0.         0.         0.375      0.         1.        ]
 [1.         0.5        0.         0.         0.        ]]
[[0.         0.5        0.         0.         0.5       ]
 [0.1875     0.         0.75       0.         0.        ]
 [0.         0.7        0.         0.7        0.        ]
 [0.         0.         0.3        0.         0.8       ]
 [1.      

[[0.         0.68152866 0.         0.         0.28025478]
 [0.32677165 0.         0.61417323 0.         0.        ]
 [0.         0.52813853 0.         0.38961039 0.        ]
 [0.         0.         0.32       0.         0.744     ]
 [0.75886525 0.19858156 0.         0.         0.        ]]
loading: [==================            ] 60.0%[[0.         0.67836257 0.         0.         0.28070175]
 [0.32075472 0.         0.61509434 0.         0.        ]
 [0.         0.5204918  0.         0.38934426 0.        ]
 [0.         0.         0.31818182 0.         0.73484848]
 [0.74172185 0.19205298 0.         0.         0.        ]]
[[0.         0.68333333 0.         0.         0.28333333]
 [0.31272727 0.         0.61454545 0.         0.        ]
 [0.         0.50980392 0.         0.38039216 0.        ]
 [0.         0.         0.31851852 0.         0.73333333]
 [0.74025974 0.19480519 0.         0.         0.        ]]
[[0.         0.68131868 0.         0.         0.28021978]
 [0.31316726 0.       

(1, 50)